In [1]:
import os 

os.chdir('../')
os.getcwd()

'c:\\Users\\arunk\\FraudDetection'

In [2]:
from fraudDetection.logger import logging
from fraudDetection.exception import FraudDetectionException
from fraudDetection.config.configuration import ConfigurationManager
from fraudDetection.components import DataIngestion, DataValidation, DataTransformation,ModelTrainer
from fraudDetection.entity import DataIngestionArtifact, DataValidationArtifact, DataTransformationArtifact, ModelTrainerArtifact

from fraudDetection.constants import CONFIG_FILE_PATH
config = ConfigurationManager(CONFIG_FILE_PATH)
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion_artifacts = DataIngestion(data_ingestion_config).initiate_data_ingestion()

data_validation_config =config.get_data_validation_config()
data_validation_artifacts =DataValidation(data_ingestion_artifacts,data_validation_config).initiate_data_validation()
data_transformation_config =config.get_data_transformation_config()
data_transformation_obj= DataTransformation(data_validation_artifacts,data_ingestion_artifacts,data_transformation_config)


c:\Users\arunk\FraudDetection\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


====>>>File already exists


In [3]:
data_transformation_obj.get_data_transfomer_object()

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('cat_pipeline',
                                                  Pipeline(steps=[('ohe_type',
                                                                   OneHotEncoder())]),
                                                  BoxList(['type'])),
                                                 ('num_pipeline',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  BoxList(['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']))])),
                ('feature_extactor', ExtraTreesRegressor())])

In [4]:
data_transformation_obj.initiate_data_transformation()

C:\Users\arunk\FraudDetection\src\fraudDetection\components\data_transformation.py:231: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.median(),inplace=True)
C:\Users\arunk\FraudDetection\src\fraudDetection\components\data_transformation.py:231: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.median(),inplace=True)


DataTransformationArtifact(is_transformed=True, message='Transformed', transformed_train_file_path='c:\\Users\\arunk\\FraudDetection\\artifacts\\transformed_dir\\train\\file_0.npz', transformed_test_file_path='c:\\Users\\arunk\\FraudDetection\\artifacts\\transformed_dir\\test\\file_0.npz', processed_object_file_path=WindowsPath('c:/Users/arunk/FraudDetection/artifacts/transformed_dir/preprocessed/preprocessed.pkl'))

In [5]:
ing_arti = data_transformation_obj.data_ingestion_artifact
val_arti = data_transformation_obj.data_validation_artifact

In [6]:
from fraudDetection.utils import save_object, read_yaml, load_data, save_numpy_array_data
from fraudDetection.constants import *
schema_path = val_arti.schema_file_path
schema = read_yaml(schema_path)
cat_col = schema[DATA_SCHEMA_CATEGORICAL_COLUMN_KEY]
col = schema[DATA_SCHEMA_COLUMNS_KEY]
num_col = schema[DATA_SCHEMA_NUMERICAL_COLUMN_KEY]
print(col)

{'step': 'int64', 'type': 'object', 'amount': 'float64', 'oldbalanceOrg': 'float64', 'newbalanceOrig': 'float64', 'oldbalanceDest': 'float64', 'newbalanceDest': 'float64', 'isFraud': 'int64'}


In [7]:
train_df = load_data(ing_arti.train_file_path,col)
test_df = load_data(ing_arti.test_file_path,col)
target_col_name = schema[DATA_SCHEMA_TARGET_COLUMN_KEY][0]

In [8]:
input_train_df = train_df.drop(target_col_name,axis=1)
target_col = train_df[target_col_name]

In [9]:
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.base import BaseEstimator, TransformerMixin

In [10]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler


In [11]:
# define the preprocessing steps for categorical and numerical columns
cat_preprocessor = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

num_preprocessor = Pipeline(steps=[
    ('scaler', RobustScaler())
])

# define the preprocessor pipeline with SMOTENC and RandomOverSampler
preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, cat_col),
    ('num', num_preprocessor, num_col)
],remainder="passthrough")


In [18]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
class SMOTENCWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features, sampling_strategy,target_feature_name):
        self.smotenc = SMOTENC(sampling_strategy=sampling_strategy, categorical_features=categorical_features)
        self.target_feature_name = target_feature_name
    def fit(self, X, y):
        self.X_resampled_, self.y_resampled_ = self.smotenc.fit_resample(X, y)
        return self

    def transform(self, X, y=None):
        df_resampled = pd.concat([pd.DataFrame(self.X_resampled_), pd.Series(self.y_resampled_, name=self.target_feature_name)], axis=1)
        return df_resampled
    
class RandomUnderSamplerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, sampling_strategy, target_feature_name):
        self.sampler = RandomUnderSampler(sampling_strategy=sampling_strategy)
        self.target_feature_name = target_feature_name
    def fit(self, X, y):
        self.X_resampled_, self.y_resampled_ = self.sampler.fit_resample(X.drop(columns=[self.target_feature_name]), X[self.target_feature_name])
        return self
    
    def transform(self, X, y=None):
        df_resampled = pd.concat([pd.DataFrame(self.X_resampled_), pd.Series(self.y_resampled_, name=self.target_feature_name)], axis=1)
        return df_resampled
    
class FeatureExtractor(BaseEstimator,TransformerMixin):
    def fit(self,X,y):
        self.model = ExtraTreesRegressor()
        self.model.fit(X,y)   
        return self
    def transform(self,X,y):
        return self

def get_data_balancing_object(target_col_name):
# define the preprocessor pipeline with SMOTENC and RandomUnderSampler
    sampling_pipeline = Pipeline(steps=[
    ('sampling', SMOTENCWrapper(categorical_features=[1], sampling_strategy=0.1,target_feature_name=target_col_name)),
    ('oversample', RandomUnderSamplerWrapper(sampling_strategy=0.5, target_feature_name= target_col_name)),
    ('preprocessor',preprocessor)
    ])
    return sampling_pipeline

sampling_pipeline = get_data_balancing_object(target_col_name=target_col_name)



Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud'],
      dtype='object')


In [20]:
resampled_df = sampling_pipeline.fit_transform(input_train_df,target_col)

Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud'],
      dtype='object')


In [22]:
resampled_df.size

143280

In [23]:
resampled = sampling_pipeline.fit(input_train_df,target_col)

Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud'],
      dtype='object')


AttributeError: 'SMOTENCWrapper' object has no attribute 'categorical_features'

AttributeError: 'SMOTENCWrapper' object has no attribute 'categorical_features'

AttributeError: 'SMOTENCWrapper' object has no attribute 'categorical_features'